In [1]:
from pyspark.sql import functions as F
from pyspark.sql.functions import udf, size, avg, count, col,sum, explode
import json

from datetime import datetime, timedelta
env = "production"
# env = "staging"



s3_client_stats_bucket = "s3://mist-secorapp-{env}/client-stats-analytics/client-stats-analytics-{env}/".format(env=env)

now = datetime.now()  - timedelta(hours=3)
date_day = now.strftime("%Y-%m-%d")
date_hour = now.strftime("%H")



# date_day= datetime.datetime.today().strftime("%Y-%m-%d")
# hr = '*'

# date_day = "2020-05-27*"
# hr = '20'

s3_client_stats_path = s3_client_stats_bucket + "dt={date}/hr={hr}/*.parquet".format(date=date_day, hr=date_hour)

print(s3_client_stats_path)




Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7,application_1591015534858_0011,pyspark3,idle,Link,Link,✔


SparkSession available as 'spark'.
s3://mist-secorapp-production/client-stats-analytics/client-stats-analytics-production/dt=2020-06-02/hr=14/*.parquet

In [2]:
df_client_stats = spark.read.parquet(s3_client_stats_path)
df_client_stats.first()

Row(ap_id='5c-5b-35-9e-bc-ac', site_id='4fcada97-23bf-4ea1-9ece-ea503c45ad6c', org_id='bbb101eb-b62d-4fb1-8c3d-030c6db7e208', firmware_version='apfw-0.5.17360-ellaria-88fe', model='AP41-US', uptime=3152855, interval=65, when=1591108939240867, radios=[], wlans=[], app_ss=[], terminator_remote_addr='', terminator_timestamp=1591108939240867, client_mac='56970b02-939c-12ec-8740-e5c1381c5643', client_wcid='56970b02-939c-12ec-8740-e5c1381c5643', client_radio_index=1, client_band='5', client_bssid='5c-5b-35-95-79-d5', client_wlan_id='', client_wlan_ssid='MEviuPuoi311', client_connected_time_sec=3633, client_inactive_time_milli_sec=62000, client_rssi=-56, client_per_antenna_rssi=[-66, -54, -49, -57], client_avg_rssi=-57, client_tx_bit_rate=780, client_tx_unicast_bytes=0, client_tx_unicast_pkts=0, client_tx_pkts_sent=0, client_tx_retries=0, client_tx_retried=0, client_tx_failed=0, client_txbps=0, client_txpps=0, client_tx_retry_exhausted=0, client_tx_rate_fall_back=0, client_rx_bit_rate=60, cli

In [3]:
df_client_stats.createOrReplaceTempView("clientStats")

df_client_stats.printSchema()

root
 |-- ap_id: string (nullable = true)
 |-- site_id: string (nullable = true)
 |-- org_id: string (nullable = true)
 |-- firmware_version: string (nullable = true)
 |-- model: string (nullable = true)
 |-- uptime: integer (nullable = true)
 |-- interval: long (nullable = true)
 |-- when: long (nullable = true)
 |-- radios: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- dev: string (nullable = true)
 |    |    |-- band: integer (nullable = true)
 |    |    |-- channel: integer (nullable = true)
 |    |    |-- secondary_channel: integer (nullable = true)
 |    |    |-- bandwidth: integer (nullable = true)
 |    |    |-- noise_floor: integer (nullable = true)
 |    |    |-- tx_power: integer (nullable = true)
 |    |    |-- max_tx_power: integer (nullable = true)
 |    |    |-- utilization: struct (nullable = true)
 |    |    |    |-- all: double (nullable = true)
 |    |    |    |-- tx: double (nullable = true)
 |    |    |    |-- rx_in_bss: dou

In [4]:
df_client_stats.show()

+-----------------+--------------------+--------------------+--------------------+-------+--------+--------+----------------+------+-----+------+----------------------+--------------------+--------------------+--------------------+------------------+-----------+-----------------+--------------+----------------+-------------------------+------------------------------+-----------+-----------------------+---------------+------------------+-----------------------+----------------------+-------------------+-----------------+-----------------+----------------+------------+------------+-------------------------+------------------------+------------------+---------------+--------------+--------------------+---------------------+-----------------+-----------------+--------------+--------------------------+------------+------------+----------------------+---------------------+-------------------+-----------------+-----------------+-----------------+--------------------+----------------+---------

In [5]:
df_client = df_client_stats.select('site_id', "client_mac", "client_ipv4")
df_client.show()


+--------------------+--------------------+---------------+
|             site_id|          client_mac|    client_ipv4|
+--------------------+--------------------+---------------+
|38d2112f-b4c3-455...|30613c03-9ed2-b4c...|    7.176.66.74|
|04d0d0ed-57a1-4ec...|c86a34e7-d596-5e4...|  10.128.147.65|
|3ba2e6fa-a60e-4e5...|a94979ff-a59c-131...|192.168.186.168|
|a7448df8-affd-481...|66f318cc-3682-4a7...|  10.108.16.100|
|59085dfb-7ead-4c6...|bf1d75d0-d050-dbb...|  10.180.127.96|
|a77944fc-0d23-4a9...|44d46227-e911-c02...|  7.119.138.183|
|209e829b-ec2e-4cd...|462636ed-93b1-055...|  10.127.217.54|
|ac4e0789-cd85-408...|6cf94279-950e-3b9...|     7.86.27.99|
|0814da38-05b6-439...|9246207b-9434-2c9...|   192.168.6.22|
|2748c237-a25e-49a...|d510be01-beca-39b...|  192.168.7.225|
|e40097cf-7b4f-458...|926af15e-3ba6-b2e...|  192.168.7.107|
|1567d1de-2168-459...|c017e848-43d8-8eb...|   192.168.6.13|
|151d6565-dfd7-415...|19736105-ad75-10f...|  7.215.234.116|
|38d2112f-b4c3-455...|ee705bb6-5608-3ea.

In [6]:
df_client_wcid = df_client_stats.select('site_id', "client_mac", "client_wcid", "client_ipv4")
df_client_wcid.show(10, False)


+------------------------------------+------------------------------------+------------------------------------+---------------+
|site_id                             |client_mac                          |client_wcid                         |client_ipv4    |
+------------------------------------+------------------------------------+------------------------------------+---------------+
|38d2112f-b4c3-4556-95a7-65ed36eed70f|30613c03-9ed2-b4cb-9027-80cd8a1200d5|30613c03-9ed2-b4cb-9027-80cd8a1200d5|7.176.66.74    |
|04d0d0ed-57a1-4ec3-9cf3-6132b24b6fa1|c86a34e7-d596-5e4f-525f-083b95f9e268|c86a34e7-d596-5e4f-525f-083b95f9e268|10.128.147.65  |
|3ba2e6fa-a60e-4e50-a3bb-2616e343fc1f|a94979ff-a59c-1311-1c7e-46d8faf1ebc7|a94979ff-a59c-1311-1c7e-46d8faf1ebc7|192.168.186.168|
|a7448df8-affd-4815-843b-c8e071699666|66f318cc-3682-4a7a-7a9c-4598a4822859|66f318cc-3682-4a7a-7a9c-4598a4822859|10.108.16.100  |
|59085dfb-7ead-4c61-a9fa-17d2b45871dc|bf1d75d0-d050-dbb9-7b25-50ebf6af2dad|bf1d75d0-d050-dbb9-7b2

In [21]:
# df_client_ip = df_client.groupBy("site_id", "client_ipv4").count()
df_client_ip = df_client.groupBy("site_id", "client_ipv4")\
            .agg(F.countDistinct('client_mac').alias('dist_clients'), F.collect_set('client_mac').alias("clients"))\
            .orderBy('dist_clients', ascending=False)



+--------------------+-----------+------------+
|             site_id|client_ipv4|dist_clients|
+--------------------+-----------+------------+
|b3c32004-a0dc-435...|    0.0.0.0|         143|
|3cf29ebc-21f1-49e...|    0.0.0.0|         108|
|28a11fdb-7d32-4ab...|    0.0.0.0|          98|
|33559aa1-1039-4a0...|    0.0.0.0|          95|
|8bc13d5c-d055-40c...|    0.0.0.0|          90|
|0bdc46d2-cefd-47e...|    0.0.0.0|          81|
|576c576e-d4d1-466...|    0.0.0.0|          75|
|5e8fe474-a9ee-4d0...|    0.0.0.0|          75|
|7bf4b471-db39-4e2...|    0.0.0.0|          69|
|c52d48f2-33f9-4bb...|    0.0.0.0|          63|
|a4b10ceb-22df-427...|    0.0.0.0|          59|
|0129e71e-cf17-4ad...|    0.0.0.0|          50|
|cad89db4-2600-468...|    0.0.0.0|          49|
|5cf9de1a-8034-447...|    0.0.0.0|          44|
|7800ed9b-40d1-434...|    0.0.0.0|          43|
|2c10bf01-8c72-483...|    0.0.0.0|          42|
|c45b99e1-969b-447...|    0.0.0.0|          41|
|64f4d304-5015-477...|    0.0.0.0|      

In [24]:
# df_client_ip.orderBy('dist_ips', ascending=False).show(10)
df_client_ip.show(10, False)

+------------------------------------+-----------+------------+
|site_id                             |client_ipv4|dist_clients|
+------------------------------------+-----------+------------+
|b3c32004-a0dc-4353-b697-1065fbd4f639|0.0.0.0    |143         |
|3cf29ebc-21f1-49e5-9956-25495fe8f41b|0.0.0.0    |108         |
|28a11fdb-7d32-4abc-b8c0-c1eda38b9f92|0.0.0.0    |98          |
|33559aa1-1039-4a0b-936d-ad096f822909|0.0.0.0    |95          |
|8bc13d5c-d055-40ce-8947-56bf5d10eff0|0.0.0.0    |90          |
|0bdc46d2-cefd-47ec-87c1-0658348009f9|0.0.0.0    |81          |
|576c576e-d4d1-466d-b9e3-96d02f7dc761|0.0.0.0    |75          |
|5e8fe474-a9ee-4d01-a2b6-b022b0f9c869|0.0.0.0    |75          |
|7bf4b471-db39-4e20-8b64-db6e93e89b4a|0.0.0.0    |69          |
|c52d48f2-33f9-4bbe-929f-31a7a5acf822|0.0.0.0    |63          |
+------------------------------------+-----------+------------+
only showing top 10 rows

In [26]:
df_client_ip.filter(col("client_ipv4")!='0.0.0.0').show(10, False)

+------------------------------------+--------------+------------+
|site_id                             |client_ipv4   |dist_clients|
+------------------------------------+--------------+------------+
|6a4513c4-d844-4a55-9b83-a649a787a7d4|192.168.40.101|11          |
|96488ea8-6dee-11e6-a43a-02e208b2d34f|192.168.1.1   |10          |
|6a4513c4-d844-4a55-9b83-a649a787a7d4|192.168.40.104|9           |
|ecd0c04f-cf77-4944-8004-0f96a4a54305|10.206.2.40   |8           |
|6a4513c4-d844-4a55-9b83-a649a787a7d4|192.168.40.107|8           |
|6a4513c4-d844-4a55-9b83-a649a787a7d4|192.168.40.103|8           |
|d52bc09e-2d96-4e05-a481-317f7ef69082|10.35.60.18   |7           |
|6a4513c4-d844-4a55-9b83-a649a787a7d4|192.168.40.106|7           |
|ecd0c04f-cf77-4944-8004-0f96a4a54305|10.206.2.39   |6           |
|6a4513c4-d844-4a55-9b83-a649a787a7d4|192.168.40.108|6           |
+------------------------------------+--------------+------------+
only showing top 10 rows

In [ ]:
# df_client_mac= df_client.groupBy("site_id", "client_mac").count()
df_client_mac = df_client.groupBy("site_id", "client_mac")\
                .agg(F.countDistinct('client_ipv4').alias("dist_ips"),
                     F.collect_set('client_ipv4').alias("IPs"))\
                .orderBy('dist_ips', ascending=False)
df_client_mac.show(10, False)

In [28]:
# df_client_mac.filter(col('dist_ips')>1).show()
df_client_mac.show(1)

+--------------------+--------------------+--------+--------------------+
|             site_id|          client_mac|dist_ips|                 IPs|
+--------------------+--------------------+--------+--------------------+
|2c10bf01-8c72-483...|f523d0cd-1d90-397...|      30|[10.10.9.164, 10....|
+--------------------+--------------------+--------+--------------------+
only showing top 1 row

In [17]:
df_client_mac.orderBy('dist_ips', ascending=False)

+--------------------+--------------------+--------+
|             site_id|          client_mac|dist_ips|
+--------------------+--------------------+--------+
|2c10bf01-8c72-483...|f523d0cd-1d90-397...|      30|
|9bbfecf4-ad7f-49b...|e9cd26d8-9401-bfb...|      25|
|eb997d93-f379-4e3...|da875cf1-37d0-5af...|      25|
|3228b76a-46da-455...|f977c85e-9e78-132...|      23|
|2c10bf01-8c72-483...|359eedd4-30a1-6a2...|      16|
|a799e12a-fc28-4ce...|fcf5308d-adf5-242...|      15|
|a799e12a-fc28-4ce...|9ebb73a9-6924-17e...|      14|
|9bbfecf4-ad7f-49b...|a1505e57-af39-827...|      14|
|9bbfecf4-ad7f-49b...|37a40225-9765-2f8...|      14|
|ef0e0e83-d0cb-4ac...|024f6fff-2c6d-31f...|      14|
+--------------------+--------------------+--------+
only showing top 10 rows

In [11]:
df_client.show()

+--------------------+--------------------+---------------+
|             site_id|          client_mac|    client_ipv4|
+--------------------+--------------------+---------------+
|38d2112f-b4c3-455...|30613c03-9ed2-b4c...|    7.176.66.74|
|04d0d0ed-57a1-4ec...|c86a34e7-d596-5e4...|  10.128.147.65|
|3ba2e6fa-a60e-4e5...|a94979ff-a59c-131...|192.168.186.168|
|a7448df8-affd-481...|66f318cc-3682-4a7...|  10.108.16.100|
|59085dfb-7ead-4c6...|bf1d75d0-d050-dbb...|  10.180.127.96|
|a77944fc-0d23-4a9...|44d46227-e911-c02...|  7.119.138.183|
|209e829b-ec2e-4cd...|462636ed-93b1-055...|  10.127.217.54|
|ac4e0789-cd85-408...|6cf94279-950e-3b9...|     7.86.27.99|
|0814da38-05b6-439...|9246207b-9434-2c9...|   192.168.6.22|
|2748c237-a25e-49a...|d510be01-beca-39b...|  192.168.7.225|
|e40097cf-7b4f-458...|926af15e-3ba6-b2e...|  192.168.7.107|
|1567d1de-2168-459...|c017e848-43d8-8eb...|   192.168.6.13|
|151d6565-dfd7-415...|19736105-ad75-10f...|  7.215.234.116|
|38d2112f-b4c3-455...|ee705bb6-5608-3ea.